# K-Nearest Neighbour für Bildsegmentierung

## Storyboard

<div style="text-align: justify">
    In den Micro Usecases TODO (Moritz CNN) und TODO (Moritz SVM) haben wir uns bereits mit Methoden der Klassifizierung beschäftigt (siehe AIAV Video <a href="https://www.youtube.com/watch?v=5dJyt0m5PJw&list=PLfJEPw9Zb0EPLEZZlNCQc9F3F7RWG6EsK&index=31">Classifier vs. Detector</a>). Die dort gezeigten Modelle erkennen ob sich ein Hammer im Sichtfeld einer Kamera befindet. Dafür wurden die Modelle zunächst anhand eines beschrifteten Datensatzes (siehe Abbildung 1), bestehend aus Bildern von Hämmern und Bildern ohne Hämmer, trainiert. Da der Trainingsdatensatz beschriftet ist, weist jedes Bild darin eine Beschriftung auf. Diese gibt an, ob sich im jeweiligen Bild ein Hammer befindet oder nicht. Das ist die sogenannte Klasse, welche wir später erkennen wollen. Nach Abschluss der Trainingsphase werden neue, unbekannte Bilder in das trainierte Modell gegeben, welches entscheidet ob im Bild ein Hammer zu sehen ist, oder nicht.
    <br /><br />
    
    Das Modell ist nach dem Training in der Lage, Bilder mit Hämmern von Bildern ohne Hämmer zu unterscheiden. Das heißt also, dass diese Modelle in der Trainingsphase  eine Grenze zwischen den Klassen in den Trainingsdaten finden. In unseren Beispielen ist diese Grenze zwischen Bildern, welche einen Hammer enthalten und Bildern ohne Hammer. Aber wie sehen solche Grenzen aus und wie werden sie gefunden?
    </div>   

<img src="images/Abbildung1Trainingsdaten.png" alt="Drawing" style="width: 600px;"/>

_Abbildung 1: Dieser beilspielhafte Trainingsdatensatz besitzt mehrere Einträge auf einer Ebene, welche einer von zwei Klassen (blaue Quadrate oder goldene Raute) angehören. Wir stellen uns die Frage, wie man in diesem Datensatz eine Grenze zwischen den Klassen finden kann, um neue, unbekannte Punkte einer der Klassen zuzuordnen._

<div style="text-align: justify">    
    Ein Verfahren, welches dieses Problem sehr einfach und zuverlässig löst, ist <a href="https://cs.nyu.edu/~roweis/papers/ncanips.pdf">k-Nearest Neighbour</a> (kNN). kNNs ermitteln die Grenzen zwischen Klassen geometrisch; die Grenze wird anhand des Abstandes zwischen benachbarten Dateneinträgen gefunden. Dafür werden beschriftete Trainingsdaten zunächst gespeichert. Soll nun ein Punkt klassifiziert werden, werden alle bekannten Datenpunkte in einem festlegbaren Radius k rund um den Punkt gezählt. Diese gezählten Einträge sind dann dei Grundlage für eine Mehrheitsentscheidung - die Klasse mit den am öftesten innerhalb des Radius vorkommenden Datenpunkten gewinnt. So Entsteht eine klare Trennung zwischen den verschiedenen Klassen basierend auf beschrifteten Trainingsdaten. Das k, also der maximale Abstand in dem Nachbarn berücksichtigt werden, ist dabei bei den meisten Implementierungen wählbar und beeinflusst das Verhalten der Klassifizierung stark.
    <br /><br />
    
    Abbildung 2 zeigt das Konzept hinter kNNs für den Datensatz in Abbildung 1 mit zwei verschiedenen k Werten. Dabei versuchen wir den zu klassifizierenden Eintrag (grüner Punkt) einer der beiden Klassen zuzuordnen. Die verschiedenen Werte für k stellen dabei den Radius um den Eintrag dar, in dem benachbarte Datenpunkte gezählt werden. Je größer das k ist, desto mehr Nachbarn werden also gezählt. Dabei sieht man die Auswirkung, welche die Wahl von k auf den Ausgang der Klassifizierung hat. Beim kleinen k wird der Eintrag Klasse 2 zugeordnet, während große Werte für k in einer Zuordnung zu Klasse 1 resultieren.
    </div>

<img src="images/Abbildung2Klassifizierung.png" alt="Drawing" style="width: 750px;"/>

_Abbildung 2: Der k-Nearest Neighbour Algorithmus erlaubt es uns neue Einträge Klassen zuzuordnen indem alle bekannten Datenpunkte in einem Bestimmten Radius gezählt werden. Die Klasse, die am öftesten innerhalb des Radius vorkommt, gewinnt. Dabei stellt k den Radius, in dem gezählt wird, dar. Für verschiedene Werte von k kann die Klassifizierung verschiedene Werte liefern._

### Praktische Implementierung

<div style="text-align: justify">
    Für die Praktische Implementierung dieses Usecases wollen wir Vorder- und Hintergrund eines Bildes trennen. Dies passiert mithilfe des kNN Algorithmus, welcher es uns wie oben beschrieben erlaubt die Grenze zwischen mehreren Klassen zu finden. Das nutzen wir, indem wir Vorder- und Hintergrund je als eigene Klasse definieren. Die Idee ist es, dann mittels kNN im Bild die Grenzen zwischen den beiden Klassen zu ermitteln, um die verschiedenen Bildbereiche dem Vorder- oder Hintergrund zuzuordnen.
    <br /><br />
    
    Diese Implementierung benötigt ein Bild vom Hintergrund, welches zunächst aufgenommen und gespeichert wird. Dieses Bild wird mit den zu analysierenden Bildern verglichen; die Unterschiede zwischen den Bildern sind dabei der Vordergrund und die Gemeinsamkeiten der Hintergrund. Verglichen wird dabei mittels des <a href="https://scikit-image.org/docs/dev/auto_examples/transform/plot_ssim.html">Structural Similarity Indexes</a>, welcher für jeden Bildpunkt einen Wert zurückgibt. Dieser Wert gibt an, wie sehr sich die beiden Bilder, also das Bild vom Hintergrund und das zu analysierende Bild, an der jeweiligen Position ähneln. Das Resultat ist ein "Wärmebild" welches angibt, wie sicher es ist, dass die Bilder unterschiedlich sind. Dieses Wärmebild wird anschließend auf eine niedrigere Auflösung skaliert, um den erforderlichen Rechenaufwand gering zu halten. Dannach setzen wir alle Werte unter einem Schwellenwert auf 0 und alle Werte über dem Schwellenwert auf 1.
     <br /><br />
     
    Grundsätzlich sind mit diesen Verarbeitungsschritten bereits alle Bildpunkte entweder dem Vorder- oder Hintergrund zugeordnet, da die resultierende Karte angibt, welcher Klasse der jeweilige Bildpunkt angehört. Einträge mit dem Wert 1 gehören zum Hintergrund, während Einträge mit dem Wert 0 zum Vordergrund gehören. Dabei weist die Karte aber oft Artefakte und unschöne Kanten um das zu erkennende Objekt auf. Diese entstehen durch sich änderende Lichtverhältnisse sowie die Verarbeitung des Bildes direkt in der Kamera. Damit wir nun schöne Grenzen zwischen Vorder- und Hintergrund erhalten und Artefakte beseitigen, verwenden wir kNN Klassifizierung. Dafür wird ein kNN Klassifikator aus der <a href="https://scikit-learn.org/stable/modules/neighbors.html">Scikit Learn Bibliothek</a> anhand der Karte trainiert. Wie oben beschrieben, werden dann die Grenzen zwischen den beiden Klassen anhand der Nachbarn jedes Eintrags berechnet. Je höher die Anzahl der Nachbarn, desto gleichmäßiger sind die berechneten Grenzen. Bei zu hoher Anzahl an Nachbarn kann es aber vorkommen, dass Details im Umriss um ein zu erkennendes Objekt weggeglättet werden. Zusätzlich dazu steigt mit wachsender Anzahl der Nachbarn auch der benötigte Rechenaufwand erheblich.
    <br /><br />
    
    Die Maske aus den von der kNN Klasse ermittelten Grenzen wird wieder auf die originale Größe der eingehenden Bilder hochskaliert und auf das eingehende Bild angewendet. Dabei werden nur als Vordergrund gekennzeichnete Teile des Bildes übernommen und als Hintergrund gekennzeichnete Teile verworfen. Damit ergibt sich ein Bild, welches nur den Vordergrund enthält. Der gesamte Ablauf der Bildverarbeitungspipeline ist in Abbildung 3 mit Visualisierungen von jedem Verarbeitungsschritt dargestellt.
    </div>

<img src="images/Abbildung3Pipeline.png" alt="Drawing" style="width: 1200px;"/>

_Abbildung 3: Um Vorder- und Hintergrund zu trennen wird das zu analysierende Bild zunächst mit einem Bild des Hintergrunds verglichen. Der aus dem Vergleich resultierende Structural Similarity Index (SSIM) wird anhand eines Schwellenwertes quantisiert. Mittels kNN werden dann Artefakte im SSIM entfernt, welche durch sich ändernde Lichtverhältnisse und Bildverarbeitung in der Kamera verursacht wurden._

### Fazit

<div style="text-align: justify">
    k-Nearest Neighor erlaubt es uns Klassifizierung anhand eines relativ kleinen Datensatzes zu betreiben, um die Grenzen zwischen Klassen zu ermitteln. Dadurch dass das Konzept hinter kNN relativ einfach ist, ist nur ein geringer Rechenaufwad für Training und Klassifizierung notwendig. Ein weiterer Vorteil von kNN ist der niedrige Notwendige Implementierungsaufwand, da diese Art von Klassifikation bereits in Form von gängigen Bibliotheken, wie <a href="https://scikit-learn.org/stable/">Scikit Learn</a> oder nativ in der <a href="https://www.rdocumentation.org/packages/class/versions/7.3-19/topics/knn">Programmiersprache R</a> verfürbar ist.
    </div>
    
<P style="page-break-before: always">

## Codedokumentation

<div style="text-align: justify">
    Die Implementierung erfolgt in <a href="https://www.python.org/downloads/">Python 3</a> mithilfe einiger Bibliotheken. Die <a href="https://opencv.org/">OpenCV</a> Implementierung für Python (<a href="https://pypi.org/project/opencv-python/">opencv-python</a>), <a href="https://pillow.readthedocs.io/en/stable/">Pillow</a> und <a href="https://scikit-image.org/">Skimage</a> stellen umfangreiche Methoden zur Bildverarbeitung zur Verfügung. Sie wurden für das Einlesen und Vorverarbeiten der Bilder verwendet. <a href="https://numpy.org/">Numpy</a> wurde für generelles Datenmanagement verwendet, während zur Klassifizierung die kNN Implementierung aus der <a href="https://scikit-learn.org/stable/">Scikit Learn Bibliothek</a> zur Verwendung kommt.
    <br /><br />
    
    Durch die Verwendung von OpenCV zum Einlesen des Eingangsbildes, ist es möglich sowohl einzelne Bilder, Videos oder direkt die Aufnahme einer Webcam zu verarbeiten. In der Dokumentation wurden einfachheitshalber nur zwei abgespeicherte Bilder miteinander verglichen.
    <br /><br />
    
    Zunächst werden alle benötigten Bibliotheken importiert.
    </div>

In [ ]:
import cv2
import numpy as np
from skimage.metrics import structural_similarity as ssim
from skimage.measure import block_reduce
from sklearn import neighbors

from PIL import Image

Dannach definieren wir Hilfsfunktionen für die k-Nearest Neighbour (kNN) Klassifizierung. _formatInput_ formatiert ein übergebenes Bild als Liste von Einträgen für Übergabe an den Klassifikator. _formatImage_ formatiert die Einträge wieder als Bild, welches mittels Pillow oder OpenCV dargestellt werden kann. _applykNN_ führt kNN Klassifikation am übergebenen Graustufenbild aus, um Artefakte aus dem quantisierten Structural Similarity Indexes zu beseitigen.

In [ ]:
def formatInput(frame):
        """ Formatiert frame als Liste von Einträgen für einfachere Klassifizierung """
        # zurückgegebenes Format: [[position_x, position_y, pixelwert_an_xy], [], [], ...]
        ret = []
        for x in range(len(frame)):
                for y in range(len(frame[0])):
                        ret.append([x, y, frame[x][y]])
        return ret

def formatImage(inp):
        """ Formatiert die Liste von Einträgen nach der Klassifizierung wieder als Bild """
        # zurückgegebenes Format: 2-dimensionales Array, compatibel mit Pillow & OpenCV
        xRes = max([entry[0] for entry in inp])+1
        yRes = max([entry[1] for entry in inp])+1
        ret = []
        for x in range(xRes):
                ret.append([0 for y in range(yRes)])
        for entry in inp:
                ret[entry[0]][entry[1]]=entry[2]
        return np.array(ret)

def applykNN(frame, K=15):
        """ Wendet kNN Klassifizierung auf ein Schwarz/Weiß Bild an, um Artefakte zu entfernen """
        # Formatierung des quantisierten SSIM Bildes
        frame = formatInput(frame)
        # Generierung der Trainingsdaten 
        # bestehend aus Positionen der Pixel und ihrer Beschriftungen
        X = np.array([[entry[0], entry[1]] for entry in frame])
        y = np.array([entry[2] for entry in frame])
        # Definition und Training des Klassifikators
        clf = neighbors.KNeighborsClassifier(K, weights='uniform')
        clf.fit(X, y)
        # Alle Pixel im Bild werden vom Klassifikator klassifiziert
        Z = clf.predict(X)
        # Formatierung und Rückabe des Bildes 
        ret = [ [X[i][0], X[i][1], Z[i]] for i in range(len(X)) ]
        return formatImage(ret)

Referenz- und Testbilder werden eingelesen, in Graustufenbilder umgewandelt und für eine schnellere Berechnung auf eine niedrigere Auflösung Skaliert. Dannach wird der Structural Similarity Index beider Bilder berechnet und noch einmal auf eine niedrigere Auflösung skaliert. Das resultierende Bild wird anhand des Schwellenwertes von 0,5 quantisiert und mittels der _applykNN_ Funktion klassifiziert. Dabei erhalten wir eine Maske, welche Vorder- und Hintergrund trennt. Die Maske wird auf die Auflösung der Eingangsbilder hochskaliert und angewendet. Dabei setzen wir alle Pixel, welche als Hintergrund klassifiziert wurden, weg. Abschließend speichern wir das Bild ab.

In [ ]:
# Einlesen und Vorbereiten von Referenzbild und zu untersuchendem Bild

def scaleImage(img, scaleFactor=2):
        """ Skaliert das Bild um scaleFactor herunter """
        width = int(len(img[0])/scaleFactor)
        height = int(len(img)/scaleFactor)
        frame = cv2.resize(img,(width, height), fx=0, fy=0, interpolation=cv2.INTER_LINEAR)
        return img

# Festelgen von Referenzbild und zu untersuchendem Bild
img1Path = './testImages/bild1.jpg'
img2Path = './testImages/bild2.jpg'

img1 = cv2.imread(img1Path)
img2 = cv2.imread(img2Path)

# Skalierung der Bilder
img1 = scaleImage(img1)
img2 = scaleImage(img2)

# Konvertierung der Bilder zu Graustufen
img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

# Ermittlung des Unterschieds zwischen Vorder- und Hintergrund
ssimRet, ssimImage = ssim(img1, img2, data_range=img2.max() - img2.min(), full=True)

# Herunterskalierung des Bildes zur Anwendung von kNN
dim = (4,4)
ssimImage = block_reduce(ssimImage, block_size=dim, func=np.mean)

# Jeder Pixel wird als Vorder- oder Hintergrund beschriftet
# Vordergrund => 0, Hintergrund => 255
threshhold = 0.5
ssimImage[ssimImage>threshhold] = 255
ssimImage[ssimImage<=threshhold] = 0

# Training und Klassifizierung mittels kNN
# So werden die Grenzen zwischen Vorder- und Hintergrund gefunden
tmp = applykNN(ssimImage, K=30)

# Hochskalierung der Maske auf die originale Auflösung
width = len(img2[0])
height =len(img2) 
tmp = cv2.resize(tmp,(width, height), fx=0, fy=0, interpolation=cv2.INTER_LINEAR)

# Hintergrund wird anhand der Maske aus dem originalen Bild entfernt
result = np.array(img2)
result[tmp==255] = 0

Image.fromarray(result).show()